# LoRA vs QLoRA vs GaLore Fine-Tuning: Practical Colab Notebook

This notebook lets you:
- Fine-tune a small LLM with **LoRA**
- Fine-tune with **QLoRA** using quantization for low VRAM
- Apply **GaLore** for efficient full-model fine-tuning
- Track VRAM usage and training speed

👋 Created for **SuperML.dev** readers.

In [ ]:
!pip install transformers peft bitsandbytes galore-torch accelerate

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print(torch.cuda.get_device_name(0))

## 🔹 LoRA Fine-Tuning Example

In [ ]:
model_name = 'tiiuae/falcon-7b-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto')

config = LoraConfig(task_type=TaskType.CAUSAL_LM, r=8, lora_alpha=16, lora_dropout=0.05)
model = get_peft_model(model, config)

model.train()
print('LoRA applied. Model ready for training.')

## 🔹 QLoRA Fine-Tuning Example

In [ ]:
model_name = 'meta-llama/Llama-2-7b-hf'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map='auto')

config = LoraConfig(task_type=TaskType.CAUSAL_LM, r=8, lora_alpha=16, lora_dropout=0.05)
model = get_peft_model(model, config)

model.train()
print('QLoRA applied. Model ready for training.')

## 🔹 GaLore Full-Model Fine-Tuning Example

In [ ]:
from galore_torch import GaLoreAdamW

optimizer = GaLoreAdamW(model.parameters(), lr=2e-5, weight_decay=0.01, rank=32, update_proj_gap=200)

# Example dummy forward + backward
input_ids = tokenizer('Hello world', return_tensors='pt').input_ids.to(model.device)
outputs = model(input_ids, labels=input_ids)
loss = outputs.loss
loss.backward()
optimizer.step()

print('GaLore optimization step completed.')

## 🔹 GPU VRAM Check

In [ ]:
!nvidia-smi

## ✅ Done

- You have now tested **LoRA**, **QLoRA**, and **GaLore** fine-tuning setups.
- Adapt these cells with your dataset and training loops as needed.

👋 Share your experiments on [r/supermldev](https://reddit.com/r/supermldev)!